
# **<center><font style="color:rgb(100,109,254)">Reconocimiento de señas en tiempo real (CNN + LSTM)</font> </center>**


## **<font style="color:rgb(134,19,348)"> Librerías e importaciones</font>**


In [ ]:
# Instalaciones
!pip install tensorflow opencv-contrib-python youtube-dl moviepy pydot graphviz
!pip install git+https://github.com/TahaAnwar/pafy.git#egg=pafy

In [2]:
# Importaciones
import os
import cv2
import pafy
import math
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
import time
import mediapipe as mp

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

## **<font style="color:rgb(134,19,348)">Preprocesamiento de datos de Entrada</font>**

Inicialización de variables

In [25]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Lista de acciones a detectar
CLASSES_LIST = ['Buenos dias', 'Buenas tardes', 'Amigo', 'Lo siento', 'Casa', 'z']

# Cantidad de videos por acción
NO_SEQUENCE = 30

# Cantidad de frames por video
SEQUENCE_LENGTH = 30

# Dirección de los datos. 
DATASET_DIR = "D:/Work/Aplicada CNN+RNN"


## **<font style="color:rgb(134,19,348)">Identificacion de Keypoints usando Mediapipe </font>**

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Inversion de color de BGR a RGB
    image.flags.writeable = False                  # Quitar propiedad de escritura
    results = model.process(image)                 # Prediccion
    image.flags.writeable = True                   # Agregar propiedad de escritura
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Inversion de color de RGB a BGR
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Dibujar puntos en el rostro
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Puntos de postura
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Puntos mano izquierda
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Puntos mano derecha

In [7]:
def draw_styled_landmarks(image, results):
    # Colocar puntos en el rostro
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Colocar puntos de postura
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Colocar puntos mano izquierda
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Colocar puntos mano derecha 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

## **<font style="color:rgb(134,19,348)">1. Modelo ConvLSTM</font>**


### **<font style="color:rgb(134,19,348)">1.1. Construcción de modelo ConvLSTM</font>**

In [26]:
def create_convlstm_model():

    model = Sequential()

    # Modelo de arquitectura.
    ########################################################################################################################
    
    model.add(ConvLSTM2D(filters = 4, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True, input_shape = (SEQUENCE_LENGTH,
                                                                                      IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 14, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    #model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(Flatten()) 
    
    model.add(Dense(len(CLASSES_LIST), activation = "softmax"))
    
    ########################################################################################################################
     
    # Summary del modelo.
    model.summary()
    
    # Retorno del modelo.
    return model

### **<font style="color:rgb(134,19,348)">1.2. Recuperar el modelo ConvLSTM</font>**

In [ ]:
convlstm_model = create_convlstm_model()
convlstm_model.load_weights('modelo_convlstm.h5')

## **<font style="color:rgb(134,19,348)">2. Modelo LRCN</font>**

### **<font style="color:rgb(134,19,348)">2.1. Construir el modelo</font>**


In [28]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()
    
    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(TimeDistributed(MaxPooling2D((4, 4)))) 
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
                                      
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))

    ########################################################################################################################

    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

### **<font style="color:rgb(134,19,348)">2.2. Recuperar el modelo LRCN</font>**


In [30]:
# Construir el modelo
LRCN_model = create_LRCN_model()

# Recuperar los pesos
LRCN_model.load_weights('LRCN-pesos.weights.h5')

## **<font style="color:rgb(134,19,348)">3. Reconocimiento en tiempo real</font>**

In [31]:
cap = cv2.VideoCapture(0)
# Setear modelo mediapipe 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    exit_flag = True
    while cap.isOpened() and exit_flag:
        frames_list = []
        for n_frame in range(SEQUENCE_LENGTH): #Para cada frame del video
            ret, frame=cap.read()
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            resized_frame = cv2.resize(image, (IMAGE_HEIGHT, IMAGE_WIDTH))  # Redimencionamiento de cada frame
            normalized_frame = resized_frame / 255                          # Normalizacion de cada frame para dar valores de 0 y 1 a cada pixel
            frames_list.append(normalized_frame)

            if len(frames_list) == SEQUENCE_LENGTH:
                # # Pass the normalized frames to the model and get the predicted probabilities.
                predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(frames_list, axis = 0))[0]

                # # Get the index of class with highest probability.qq
                predicted_label = np.argmax(predicted_labels_probabilities)

                # # Get the class name using the retrieved index.
                predicted_class_name = CLASSES_LIST[predicted_label]

                # Visualizacion
                cv2.rectangle(image, (0,0), (250, 40), (245, 117, 16), -1)
                cv2.putText(image, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Mostrar en pantalla
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(1000)

            else:
                cv2.putText(image, 'Recolectando frames....', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Mostrar en pantalla
                cv2.imshow('OpenCV Feed', image)
                
            if cv2.waitKey(10) & 0xFF == ord('q'):
                exit_flag = False
                break
        
    cap.release()
    cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
